In [1]:
import numpy as np
from numpy.linalg import norm
from numpy.random import normal, beta, choice, permutation
from numpy.typing import NDArray
from scipy.special import gamma, digamma
from typing import Callable

# Стохастический градиентный спуск RMSProp

In [2]:
def SGD_RMSProp(
    start: NDArray,
    X: NDArray,
    y: NDArray,
    L_grad: Callable,
    batch_size: int | float,
    L: Callable | None = None,
    learning_rate: float = 0.01,
    decay_rate: float = 0.5,
    use_epoch: bool = False,
    max_iter=1000,
    tol=1e-7,
    n_iter_no_change: int = 5,
    **kwargs
) -> dict:
    curr_point = start
    min_error = None
    run_avg = np.zeros(np.size(start))
    curr_iter = 0
    curr_value = None
    n = X.shape[0]
    if L is not None:
        L_last = None
    if use_epoch:
        curr_epoch = 0

    if type(batch_size) is float and batch_size < 1:
        batch_size = n * batch_size

    batch_size = int(batch_size)

    while min_error is None or (curr_iter < max_iter and min_error >= tol):
        if use_epoch:
            idx = permutation(n)
            X_perm, y_perm = X[idx], y[idx]

            if L is not None:
                if curr_epoch == 0:
                    L_start = L(curr_point, X, y, **kwargs)
                    L_last = L_start
                elif curr_epoch % n_iter_no_change == 0 and curr_epoch != 0:
                    if np.abs(L_last - L_start) < tol:
                        learning_rate /= 5
                    L_last = L_start

            for batch_start in range(0, n, batch_size):
                batch_end = batch_start + batch_size
                batch_X, batch_y = (
                    X_perm[batch_start:batch_end],
                    y_perm[batch_start:batch_end],
                )
                curr_grad = L_grad(curr_point, batch_X, batch_y, **kwargs)

                run_avg = decay_rate * run_avg + (1 - decay_rate) * curr_grad**2

                curr_point -= learning_rate / np.sqrt(run_avg) * curr_grad
                curr_iter += 1

            W_error = norm(learning_rate * curr_grad)
            if L is not None:
                L_new = L(curr_point, X, y, **kwargs)
                L_error = np.abs(L_start - L_new)
                L_start = min(L_new, L_start)
                min_error = min(W_error, L_error)
            else:
                min_error = W_error

            curr_epoch += 1

        else:
            idx = choice(n, batch_size, replace=False)
            batch_X, batch_y = X[idx], y[idx]
            if L is not None:
                L_start = L(curr_point, X, y, **kwargs)

            curr_grad = L_grad(curr_point, batch_X, batch_y, **kwargs)
            run_avg = decay_rate * run_avg + (1 - decay_rate) * curr_grad**2

            curr_point -= learning_rate / np.sqrt(run_avg) * curr_grad

            W_error = norm(learning_rate * curr_grad)
            if L is not None:
                L_error = np.abs(L_start - L(curr_point, X, y, **kwargs))
                min_error = min(W_error, L_error)
            else:
                min_error = W_error

            curr_iter += 1

    if L is not None:
        curr_value = L(curr_point, X, y, **kwargs)

    return {
        "point": curr_point,
        "L_value": curr_value,
        "grad_value": curr_grad,
        "iterations": curr_iter,
    }


### Тест SGD RMSPror

In [3]:
def L(w, X, y):
    return norm(X.dot(w) - y) ** 2 / y.size


def L_grad(w, X, y):
    return 2 * X.T.dot(X.dot(w) - y) / y.size


np.random.seed(42)
nrow, ncol = 500, 4
X = normal(0, 1, ncol * nrow).reshape(nrow, ncol)
X_ones = np.hstack([X, np.ones((nrow, 1))])
true_w = np.array([2, -3, 1, 0.5, 4])
y = X_ones.dot(true_w) + normal(0, 1, nrow)
w_start = normal(0, 1, ncol + 1)

sgd_rmsprop_res = SGD_RMSProp(
    start=w_start,
    X=X_ones,
    y=y,
    L_grad=L_grad,
    batch_size=100,
    decay_rate=0.9
)

In [4]:
print(f'Iterations: {sgd_rmsprop_res["iterations"]}')
print(f'w_e: {sgd_rmsprop_res["point"]}')
print(f'||w_e-w_t||^2 = {norm(sgd_rmsprop_res["point"] - true_w) ** 2}')

Iterations: 1000
w_e: [ 1.98456237 -3.02617223  0.91354794  0.52762237  3.98789091]
||w_e-w_t||^2 = 0.009306890328730593


## Функция правдоподобия для бета-регрессии
[Статья про бета-регрессию](https://www.ime.usp.br/~sferrari/beta.pdf)

Пусть $\xi\sim \Beta(\alpha, \beta)$.

**Плотность**: 
$$
    f_\xi(x)=\frac{x^{\alpha-1}(1-x)^{\beta-1}}{\Beta(\alpha, \beta)}, \qquad x \in (0, 1).
$$
Для построения бета-регрессии удобнее работать в параметризации через среднее и "точность":
$$
    \mu = \frac{\alpha}{\alpha + \beta}, \qquad \varphi = \alpha + \beta,\\
    \mu \in (0, 1), \qquad \varphi > 0.
$$
Тогда старые параметры выражаются следующим образом:
$$
    \alpha = \mu\varphi, \qquad \beta = (1-\mu)\varphi.
$$ 

Среднее и дисперсия хорошо выражаются через новые параметры:
$$
    \mathrm{E}(\xi) = \mu, \qquad \sigma^2 = \frac{\mu(1-\mu)}{1 + \varphi}
$$

**Плотность в новой параметризации**:
$$
    f_\xi(x) = \frac{\Gamma(\varphi)}{\Gamma(\mu\varphi)\Gamma((1-\mu)\varphi)}x^{\mu\varphi-1}(1-x)^{(1-\mu)\varphi-1}, \qquad x \in (0, 1).
$$

Пусть $\mathbf{X}\in \mathbb{R}^{n\times p}$ - выборка регрессоров, $Y \in \mathbb{R}^{n}$ - выборка откликов.
Предполагается, что $y_i \sim \Beta(\mu_i, \varphi)$, где параметр $\varphi$ неизвестен, а
$\mu_i$ выражается через регрессоры:
$$
    g(\mu_i) = \bold{x}^{\mathrm{T}}_i \boldsymbol{\beta}.
$$
$g(t)$ - произвольная линк-функция, например логит:
$$
    g(\mu_i) = \log\left( \frac{\mu_i}{1-\mu_i} \right) = \bold{x}^{\mathrm{T}}_i \boldsymbol{\beta} \implies 
    \mu_i = \frac{e^{\bold{x}^{\mathrm{T}}_i \boldsymbol{\beta}}}{1 + e^{\bold{x}^{\mathrm{T}}_i \boldsymbol{\beta}}}.
$$

**Логарифм функции правдоподобия**:
$$
    L(\mathbf{X}, \beta, \varphi; Y) = \frac{1}{n}\sum_{i=1}^{n}l(\mu_i(\bold{x}_i, \beta), \varphi; y_i)\\

    l(\mu_i(\bold{x}_i, \beta), \varphi; y_i) = \log\Gamma(\varphi) - \log\Gamma(\mu_i\varphi) - \log\Gamma((1-\mu_i)\varphi) + 
    (\mu_i\varphi - 1) \log y_i + ((1 - \mu_i)\varphi - 1)\log (1 - y_i).
$$

## Градиент логарифма функции правдободобия
Пусть $\mathbf{X}$ и $Y$ фиксированы, обозначим 
$$y_i^* = \log(y_i / (1 - y_i)), \qquad \mu_i^*=\psi(\mu_i\varphi) - \psi((1 - \mu_i)\varphi), \qquad
\mathbf{T} = \mathrm{diag}\left( 1 / g'(\mu_1), \ldots, 1 / g'(\mu_n) \right),
$$
$$
  Y^* = (y_1^*, \ldots, y_n^*)^{\mathrm{T}}, \qquad \boldsymbol{\mu}^* = (\mu_1^*, \ldots, \mu_n^*)^{\mathrm{T}},
$$
где $\psi(z) = (\log\Gamma(z))'$ - дигамма-функция (есть в модуле `scipy.special`),
тогда градиент логарифма функции правдоподобия равен 
$\nabla L(\beta, \varphi) = \left( L_{\beta}^{\mathrm{T}}(\beta, \varphi), L_{\varphi}(\beta, \varphi) \right)^{\mathrm{T}}$, где
$$
    L_{\beta}(\beta, \varphi) = \varphi \mathbf{X}^{\mathrm{T}}\mathbf{T}(Y^* - \boldsymbol{\mu^*}),
$$
$$
  L_{\varphi}(\beta, \varphi) = \sum_{i=1}^n 
  \big(  
    \mu_i(y_i^* - \mu_i^*) + \log(1 - y_i) - \psi((1 - \mu_i)\varphi) + \psi(\varphi)
  \big).
$$

In [ ]:
def logit_inverse(x: NDArray, beta: NDArray) -> NDArray:
    t = np.exp(x.dot(beta))
    return t / (1 + t)


def logit_deriv(mu: NDArray) -> NDArray:
    return 1 / (mu - mu * mu)


def beta_log_likelihood(
    parameters: NDArray,
    X: NDArray,
    y: NDArray,
    link_inverse: Callable[[NDArray, NDArray], float],
    link_deriv=None,
) -> float:
    beta, phi = parameters[:-1], parameters[-1]
    mu = link_inverse(X, beta)

    prod = mu * phi
    return np.sum(
        np.log(gamma(phi))
        - np.log(gamma(prod))
        - np.log(gamma(phi - prod))
        + (prod - 1) * np.log(y)
        + (phi - prod - 1) * np.log(1 - y)
    )


def beta_inv_log_likelihood(
    parameters: NDArray,
    X: NDArray,
    y: NDArray,
    link_inverse: Callable[[NDArray, NDArray], float],
    link_deriv=None,
) -> float:
    return -beta_log_likelihood(parameters, X, y, link_inverse, link_deriv)


def beta_illh_grad(
    parameters: NDArray,
    X: NDArray,
    Y: NDArray,
    link_inverse: Callable[[NDArray, NDArray], NDArray],
    link_deriv: Callable[[NDArray], NDArray],
) -> NDArray:
    beta, phi = parameters[:-1], parameters[-1]
    mu_vec = link_inverse(X, beta)
    Y_star = np.log(Y / (1 - Y))
    mu_star = digamma(mu_vec * phi) - digamma((1 - mu_vec) * phi)
    T_mat = np.diag(1 / link_deriv(mu_vec))
    L_beta = phi * X.T.dot(T_mat).dot(Y_star - mu_star)
    L_phi = np.sum(
        mu_vec * (Y_star - mu_star)
        + np.log(1 - Y)
        - digamma((1 - mu_vec) * phi)
        + digamma(phi)
    )
    return -np.append(L_beta, L_phi)

### Тест SGD RMSProp для бета регрессии

In [ ]:
%%time

np.random.seed(42)
nrow, ncol = 500, 4
X = normal(0, 1, ncol * nrow).reshape(nrow, ncol)
X_ones = np.hstack([X, np.ones((nrow, 1))])
true_beta = np.array([0.1, 0.3, 0.01, 0.5, 0.4])
true_phi = 3
true_mu = logit_inverse(X_ones, true_beta)
y = np.array([beta(true_phi * mu, (1 - mu) * true_phi) for mu in true_mu])
w_start = np.append(normal(0, 1, ncol + 1), 2)

beta_res = SGD_RMSProp(
    start=w_start,
    X=X_ones,
    y=y,
    L_grad=beta_illh_grad,
    L=beta_inv_log_likelihood,
    batch_size=50,
    learning_rate=0.01,
    decay_rate=0.1,
    max_iter=10000,
    tol=1e-4,
    link_inverse=logit_inverse,
    link_deriv=logit_deriv,
)

CPU times: user 89.3 ms, sys: 1.96 ms, total: 91.3 ms
Wall time: 90.7 ms


In [35]:
print(f'Iterations: {beta_res["iterations"]}')
print(f'w_e = {beta_res["point"]}')
print(f'||w_e-w_t||^2 = {norm(beta_res["point"] - np.append(true_beta, true_phi)) ** 2}')

Iterations: 407
w_e = [0.10663241 0.33138512 0.01614833 0.50256236 0.46656616 3.05184792]
||w_e-w_t||^2 = 0.008192642322184963
